Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/shared/homes/sguan/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [209]:
# pickle_file = './support_data/notMNIST.pickle'
pickle_file = './support_data/notMNIST_clean.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (195125, 28, 28) (195125,)
Validation set (9828, 28, 28) (9828,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [210]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (195125, 28, 28, 1) (195125, 10)
Validation set (9828, 28, 28, 1) (9828, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [211]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [63]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [64]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.944964
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%


Minibatch loss at step 50: 1.935623
Minibatch accuracy: 18.8%
Validation accuracy: 38.2%
Minibatch loss at step 100: 1.414533
Minibatch accuracy: 68.8%


Validation accuracy: 65.3%
Minibatch loss at step 150: 0.438555
Minibatch accuracy: 93.8%


Validation accuracy: 77.2%
Minibatch loss at step 200: 0.438523
Minibatch accuracy: 93.8%


Validation accuracy: 77.4%
Minibatch loss at step 250: 1.545897
Minibatch accuracy: 62.5%


...

Minibatch loss at step 850: 0.452454
Minibatch accuracy: 87.5%
Validation accuracy: 82.6%


Minibatch loss at step 900: 0.238688
Minibatch accuracy: 93.8%
Validation accuracy: 82.7%


Minibatch loss at step 950: 0.095232
Minibatch accuracy: 100.0%
Validation accuracy: 83.1%


Minibatch loss at step 1000: 0.817637
Minibatch accuracy: 81.2%
Validation accuracy: 82.9%


Test accuracy: 89.5%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [21]:
num_channels

1

In [117]:
# write the original model from scratch

batch_size = 16
M_conv_p = 5  # patch size 
M_conv_d = 16 # depth
M_conv_s = 2  # stride
M_full = 64
lambda_l2 = 0.001   # parameter for L2 penalty

M1_conv_shape = np.array([image_size//M_conv_s, image_size//M_conv_s, M_conv_d])
M2_conv_shape = np.array([image_size//M_conv_s**2, image_size//M_conv_s**2, M_conv_d])
M2_conv_size = np.prod(M2_conv_shape)


graph = tf.Graph()

with graph.as_default():
    
    
    """ declare placeholder, constant and variable """
    # placeholder
    X_tr = tf.placeholder(dtype=tf.float32, shape=[batch_size, image_size, image_size, num_channels])
    Y_tr = tf.placeholder(dtype=tf.float32, shape=[batch_size, num_labels])
    
    # constant
    X_vl = tf.constant(valid_dataset, dtype=tf.float32)
    X_ts = tf.constant( test_dataset, dtype=tf.float32)
    
    # variable
    W1_conv = tf.Variable(tf.truncated_normal(shape=[M_conv_p, M_conv_p, num_channels, M_conv_d], stddev=0.05))
    b1_conv = tf.Variable(tf.zeros(M_conv_d))
    W2_conv = tf.Variable(tf.truncated_normal(shape=[M_conv_p, M_conv_p, M_conv_d, M_conv_d], stddev=0.05))
    b2_conv = tf.Variable(tf.zeros(M_conv_d))
    W3_full = tf.Variable(tf.truncated_normal(shape=[M2_conv_size, M_full], stddev=0.05))
    b3_full = tf.Variable(tf.zeros(M_full))
    W4_full = tf.Variable(tf.truncated_normal(shape=[M_full, num_labels], stddev=0.05))
    b4_full = tf.Variable(tf.zeros(num_labels))

    """ define the neural network model """
    def model(X_in):
        X1_conv = tf.nn.relu(b1_conv + \
                             tf.nn.conv2d(X_in, filter=W1_conv, strides=[1, M_conv_s, M_conv_s, 1], padding='SAME'))
        X2_conv = tf.nn.relu(b2_conv + \
                             tf.nn.conv2d(X1_conv, filter=W2_conv, strides=[1, M_conv_s, M_conv_s, 1], padding='SAME'))
        X2_conv_flat = tf.reshape(X2_conv, [-1, M2_conv_size])
        X3_full = tf.nn.relu(b3_full + tf.matmul(X2_conv_flat, W3_full))
        X_out = b4_full + tf.matmul(X3_full, W4_full)
        return X_out

    """ define training loss and optimizer """
    logits = model(X_tr)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_tr, logits=logits)) +\
           lambda_l2*tf.nn.l2_loss(W1_conv) + lambda_l2*tf.nn.l2_loss(W2_conv) + \
           lambda_l2*tf.nn.l2_loss(W3_full) + lambda_l2*tf.nn.l2_loss(W4_full)
    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    """ get train/valid/test predictions """
    y_hat_tr = tf.nn.softmax(logits)
    y_hat_vl = tf.nn.softmax( model(X_vl) )
    y_hat_ts = tf.nn.softmax( model(X_ts) )

In [121]:
# write the model using max pooling

batch_size = 16
M_conv_p = 5  # patch size 
M_conv_d = 16 # depth
M_pool_s = 2  # pooling stride
M_pool_k = 2  # pooling kernel size
M_full = 64
lambda_l2 = 0.001   # parameter for L2 penalty

M1_conv_shape = np.array([image_size//M_pool_s, image_size//M_pool_s, M_conv_d])
M2_conv_shape = np.array([image_size//M_pool_s**2, image_size//M_pool_s**2, M_conv_d])
M2_conv_size = np.prod(M2_conv_shape)


graph = tf.Graph()

with graph.as_default():
    """ declare placeholder, constant and variable """
    # placeholder
    X_tr = tf.placeholder(dtype=tf.float32, shape=[batch_size, image_size, image_size, num_channels])
    Y_tr = tf.placeholder(dtype=tf.float32, shape=[batch_size, num_labels])
    
    # constant
    X_vl = tf.constant(valid_dataset, dtype=tf.float32)
    X_ts = tf.constant( test_dataset, dtype=tf.float32)
    
    # variable
    W1_conv = tf.Variable(tf.truncated_normal(shape=[M_conv_p, M_conv_p, num_channels, M_conv_d], stddev=0.05))
    b1_conv = tf.Variable(tf.zeros(M_conv_d))
    W2_conv = tf.Variable(tf.truncated_normal(shape=[M_conv_p, M_conv_p, M_conv_d, M_conv_d], stddev=0.05))
    b2_conv = tf.Variable(tf.zeros(M_conv_d))
    W3_full = tf.Variable(tf.truncated_normal(shape=[M2_conv_size, M_full], stddev=0.05))
    b3_full = tf.Variable(tf.zeros(M_full))
    W4_full = tf.Variable(tf.truncated_normal(shape=[M_full, num_labels], stddev=0.05))
    b4_full = tf.Variable(tf.zeros(num_labels))

    """ define the neural network model """
    def model(X_in):
        X1_conv = tf.nn.relu(b1_conv + \
                             tf.nn.conv2d(X_in,    filter=W1_conv, strides=[1, 1, 1, 1], padding='SAME'))
        X1_pool = tf.nn.max_pool(X1_conv, ksize=[1,M_pool_k, M_pool_k, 1], strides=[1,M_pool_s, M_pool_s, 1], padding='SAME')
        X2_conv = tf.nn.relu(b2_conv + \
                             tf.nn.conv2d(X1_pool, filter=W2_conv, strides=[1, 1, 1, 1], padding='SAME'))
        X2_pool = tf.nn.max_pool(X2_conv, ksize=[1,M_pool_k, M_pool_k, 1], strides=[1,M_pool_s, M_pool_s, 1], padding='SAME')
        X2_conv_flat = tf.reshape(X2_pool, [-1, M2_conv_size])
        X3_full = tf.nn.relu(b3_full + tf.matmul(X2_conv_flat, W3_full))
        X_out = b4_full + tf.matmul(X3_full, W4_full)
        return X_out

    """ define training loss and optimizer """
    logits = model(X_tr)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_tr, logits=logits)) +\
           lambda_l2*tf.nn.l2_loss(W1_conv) + lambda_l2*tf.nn.l2_loss(W2_conv) + \
           lambda_l2*tf.nn.l2_loss(W3_full) + lambda_l2*tf.nn.l2_loss(W4_full)
    
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    """ get train/valid/test predictions """
    y_hat_tr = tf.nn.softmax(logits)
    y_hat_vl = tf.nn.softmax( model(X_vl) )
    y_hat_ts = tf.nn.softmax( model(X_ts) )

In [124]:
# make a session to run the model

num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()    # remember to initialize variables
    print('model variable initialized')
    for step in range(num_steps):
        offset = (step*batch_size)%(len(train_dataset)-batch_size)
        feed_dict = {X_tr: train_dataset[offset:offset+batch_size], Y_tr:  train_labels[offset:offset+batch_size]}
        [_, loss_cur, y_hat_tr_cur] = session.run([optimizer, loss, y_hat_tr], feed_dict=feed_dict)
        if step%100 == 0:
            acc_tr = accuracy(y_hat_tr_cur, train_labels[offset:offset+batch_size])
            acc_vl = accuracy(y_hat_vl.eval(), valid_labels)
            print('step={:>6}, loss={:>6f},  accuracy: tr={:6.2f}, vl={:6.2f}'.format(step, loss_cur, acc_tr, acc_vl))
    acc_ts = accuracy(y_hat_ts.eval(),  test_labels)
    print('finally, loss={:>6f},  accuracy: vl={:6.2f}, test={:6.2f}'.format(loss_cur, acc_vl, acc_ts))

model variable initialized


step=     0, loss=2.362159,  accuracy: tr=  6.25, vl= 11.57


step=   100, loss=2.244925,  accuracy: tr= 43.75, vl= 31.38


step=   200, loss=0.622760,  accuracy: tr= 87.50, vl= 72.80


step=   300, loss=1.211799,  accuracy: tr= 62.50, vl= 76.76


...

step=   700, loss=0.580157,  accuracy: tr= 87.50, vl= 82.51


step=   800, loss=0.729119,  accuracy: tr= 87.50, vl= 81.74


step=   900, loss=0.312599,  accuracy: tr= 93.75, vl= 83.42


step=  1000, loss=0.746063,  accuracy: tr= 81.25, vl= 83.71


finally, loss=0.746063,  accuracy: vl= 83.71, test= 90.43


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [229]:
# write the model using max pooling

batch_size = 64
M_conv_p = 5  # patch size 
M_conv1_d = 6  # depth
M_conv2_d = 16 # depth
M_pool_s = 2  # pooling stride
M_pool_k = 2  # pooling kernel size
M_full3 = 120
M_full4 = 84
lambda_l2 = 0.002   # parameter for L2 penalty


def size_after(size_in, stride=1, kernel=1, padding='SAME'):
    """ tool function to calculate the size of next layer after convolution or pooling """
    devident, remainder =  divmod(size_in-(padding=='VALID')*(kernel-1), stride)
    return devident + (remainder>0)


size_M1_conv = size_after(image_size, kernel=M_conv_p, padding='SAME')
size_M1_pool = size_after(size_M1_conv, stride=M_pool_s ,kernel=M_pool_k, padding='VALID')
size_M2_conv = size_after(size_M1_pool, kernel=M_conv_p, padding='VALID')
size_M2_pool = size_after(size_M2_conv, stride=M_pool_s, kernel=M_pool_k, padding='VALID')

M2_size = size_M2_pool*size_M2_pool*M_conv2_d


graph = tf.Graph()

with graph.as_default():
    # global step for rate decay
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.2, global_step, decay_steps=10000, decay_rate=0.1)
    
    """ declare placeholder, constant and variable """
    # placeholder
    X_tr = tf.placeholder(dtype=tf.float32, shape=[batch_size, image_size, image_size, num_channels])
    Y_tr = tf.placeholder(dtype=tf.float32, shape=[batch_size, num_labels])
    
    # constant
    X_vl = tf.constant(valid_dataset, dtype=tf.float32)
    X_ts = tf.constant( test_dataset, dtype=tf.float32)
    
    # variable
    W1_conv = tf.Variable(tf.truncated_normal(shape=[M_conv_p, M_conv_p, num_channels, M_conv1_d], stddev=0.05))
    b1_conv = tf.Variable(tf.zeros(M_conv1_d))
    W2_conv = tf.Variable(tf.truncated_normal(shape=[M_conv_p, M_conv_p, M_conv1_d, M_conv2_d], stddev=0.05))
    b2_conv = tf.Variable(tf.zeros(M_conv2_d))
    W3_full = tf.Variable(tf.truncated_normal(shape=[M2_size, M_full3], stddev=0.05))
    b3_full = tf.Variable(tf.zeros(M_full3))
    W4_full = tf.Variable(tf.truncated_normal(shape=[M_full3, M_full4], stddev=0.05))
    b4_full = tf.Variable(tf.zeros(M_full4))
    W5_full = tf.Variable(tf.truncated_normal(shape=[M_full4, num_labels], stddev=0.05))
    b5_full = tf.Variable(tf.zeros(num_labels))

    """ define the neural network model """
    def model(X_in, dropout_keep_prob=1.0):
        X1_conv = tf.nn.relu(b1_conv + \
                             tf.nn.conv2d(X_in,    filter=W1_conv, strides=[1, 1, 1, 1], padding='SAME'))
        X1_pool = tf.nn.max_pool(X1_conv, ksize=[1,M_pool_k, M_pool_k, 1], strides=[1,M_pool_s, M_pool_s, 1], padding='SAME')
        X1_drop = tf.nn.dropout(X1_pool, keep_prob=dropout_keep_prob)
        X2_conv = tf.nn.relu(b2_conv + \
                             tf.nn.conv2d(X1_drop, filter=W2_conv, strides=[1, 1, 1, 1], padding='VALID'))
        X2_pool = tf.nn.max_pool(X2_conv, ksize=[1,M_pool_k, M_pool_k, 1], strides=[1,M_pool_s, M_pool_s, 1], padding='SAME')
        X2_drop = tf.nn.dropout(X2_pool, keep_prob=dropout_keep_prob)
        X2_flat = tf.reshape(X2_drop, [-1, M2_size])
        X3_full = tf.nn.relu(b3_full + tf.matmul(X2_flat, W3_full))
        X3_drop = tf.nn.dropout(X3_full, keep_prob=dropout_keep_prob)
        X4_full = tf.nn.relu(b4_full + tf.matmul(X3_drop, W4_full))
        X4_drop = tf.nn.dropout(X4_full, keep_prob=dropout_keep_prob)
        X_out = b5_full + tf.matmul(X4_drop, W5_full)
        return X_out

    """ define training loss and optimizer """
    logits = model(X_tr, dropout_keep_prob=0.80)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_tr, logits=logits)) +\
           lambda_l2*tf.nn.l2_loss(W1_conv) + lambda_l2*tf.nn.l2_loss(W2_conv) + \
           lambda_l2*tf.nn.l2_loss(W3_full) + lambda_l2*tf.nn.l2_loss(W4_full)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    """ get train/valid/test predictions """
    y_hat_tr = tf.nn.softmax( model(X_tr, dropout_keep_prob=1.0))
    y_hat_vl = tf.nn.softmax( model(X_vl, dropout_keep_prob=1.0))
    y_hat_ts = tf.nn.softmax( model(X_ts, dropout_keep_prob=1.0))


# make a session to run the model
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()    # remember to initialize variables
    print('model variable initialized')
    for step in range(num_steps):
        offset = (step*batch_size)%(len(train_dataset)-batch_size)
        if offset<batch_size:    # shuffle the dataset after going over it once
            indx_shuffle = np.random.permutation(len(train_dataset))
            train_dataset = train_dataset[indx_shuffle]
            train_labels  = train_labels[indx_shuffle]
        feed_dict = {X_tr: train_dataset[offset:offset+batch_size], Y_tr:  train_labels[offset:offset+batch_size]}
        [_, loss_cur, y_hat_tr_cur] = session.run([optimizer, loss, y_hat_tr], feed_dict=feed_dict)
        if step%100 == 0:
            acc_tr = accuracy(y_hat_tr_cur, train_labels[offset:offset+batch_size])
            acc_vl = accuracy(y_hat_vl.eval(), valid_labels)
            print('step={:>6}, loss={:>6f},  accuracy: tr={:6.2f}, vl={:6.2f}'.format(step, loss_cur, acc_tr, acc_vl))
    acc_ts = accuracy(y_hat_ts.eval(),  test_labels)
    print('finally, loss={:>6f},  accuracy: vl={:6.2f}, test={:6.2f}'.format(loss_cur, acc_vl, acc_ts))

print('finished')

model variable initialized


step=     0, loss=2.419630,  accuracy: tr= 10.94, vl= 10.10


step=   100, loss=2.361379,  accuracy: tr= 29.69, vl= 28.58


step=   200, loss=1.210154,  accuracy: tr= 71.88, vl= 71.51


step=   300, loss=0.957242,  accuracy: tr= 75.00, vl= 79.57


...

step=  9600, loss=0.437496,  accuracy: tr= 92.19, vl= 89.72


step=  9700, loss=0.279643,  accuracy: tr= 98.44, vl= 89.68


step=  9800, loss=0.419997,  accuracy: tr= 90.62, vl= 89.92


step=  9900, loss=0.475109,  accuracy: tr= 90.62, vl= 89.90


step= 10000, loss=0.383266,  accuracy: tr= 93.75, vl= 89.82
finally, loss=0.383266,  accuracy: vl= 89.82, test= 95.49
finished


In [207]:
np.random.permutation(10)

array([2, 1, 7, 9, 5, 3, 8, 6, 0, 4])